# Import

In [84]:
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F 
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn as nn

# Download and Explore the data

In [2]:
DATASET_URL = "https://gist.github.com/BirajCoder/5f068dfe759c1ea6bdfce9535acdb72d/raw/c84d84e3c80f93be67f6c069cbdc0195ec36acbd/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, ".")

83.0%

101.9%


In [35]:
df = pd.read_csv("insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [36]:
num_rows = df.shape[0]
print(num_rows)

1338


In [37]:
num_cols = df.shape[1]
print(num_cols)

7


In [45]:
input_cols = ["age", "sex", "bmi", "children", "smoker", "region"]
input_cols

['age', 'sex', 'bmi', 'children', 'smoker', 'region']

In [46]:
# Which of the input columns are non-numeric or categorial variables ?
categorical_cols = ["sex", "smoker", "region"]
categorical_cols

['sex', 'smoker', 'region']

In [47]:
output_cols = ["charges"]
output_cols

['charges']

# Prepare the dataset for training

In [111]:
def dataframe_to_tensor(dataframe):
    # Make a copy
    df = dataframe.copy()
    # Convert non-numerical columns to number
    for col in categorical_cols:
        df[col] = df[col].astype("category").cat.codes
    # Extract input & optput as numpy array
    inputs_array = df[input_cols].to_numpy().astype(np.float32)
    targets_array = df[output_cols].to_numpy().astype(np.float32)

    return inputs_array, targets_array

In [112]:
# Convert df to np array
inputs_array, outputs_array = dataframe_to_tensor(df)

In [113]:
inputs_array

array([[19.  ,  0.  , 27.9 ,  0.  ,  1.  ,  3.  ],
       [18.  ,  1.  , 33.77,  1.  ,  0.  ,  2.  ],
       [28.  ,  1.  , 33.  ,  3.  ,  0.  ,  2.  ],
       ...,
       [18.  ,  0.  , 36.85,  0.  ,  0.  ,  2.  ],
       [21.  ,  0.  , 25.8 ,  0.  ,  0.  ,  3.  ],
       [61.  ,  0.  , 29.07,  0.  ,  1.  ,  1.  ]], dtype=float32)

In [114]:
outputs_array

array([[16884.924 ],
       [ 1725.5522],
       [ 4449.462 ],
       ...,
       [ 1629.8335],
       [ 2007.945 ],
       [29141.36  ]], dtype=float32)

In [115]:
# Convert np to tensor
inputs = torch.from_numpy(inputs_array)
targets = torch.from_numpy(outputs_array)

In [116]:
print(inputs, inputs.shape)
print(targets, targets.shape)

tensor([[19.0000,  0.0000, 27.9000,  0.0000,  1.0000,  3.0000],
        [18.0000,  1.0000, 33.7700,  1.0000,  0.0000,  2.0000],
        [28.0000,  1.0000, 33.0000,  3.0000,  0.0000,  2.0000],
        ...,
        [18.0000,  0.0000, 36.8500,  0.0000,  0.0000,  2.0000],
        [21.0000,  0.0000, 25.8000,  0.0000,  0.0000,  3.0000],
        [61.0000,  0.0000, 29.0700,  0.0000,  1.0000,  1.0000]]) torch.Size([1338, 6])
tensor([[16884.9238],
        [ 1725.5522],
        [ 4449.4619],
        ...,
        [ 1629.8335],
        [ 2007.9449],
        [29141.3594]]) torch.Size([1338, 1])


In [117]:
# Create PyTorch dataset
dataset = TensorDataset(inputs, targets)
print(dataset[0])

(tensor([19.0000,  0.0000, 27.9000,  0.0000,  1.0000,  3.0000]), tensor([16884.9238]))


In [118]:
# Split train and validation
val_percent = 0.2
val_size = int(num_rows * 0.2)
train_size = num_rows - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
print(len(train_ds), len(val_ds))

1071 267


In [119]:
# Batch size
batch_size = 32
# Data Loader
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=batch_size)

In [120]:
# Check each batch of data
for xb, yb in train_dl:
    print(f"Inputs: {xb}")
    print(f"Targets: {yb}")
    break

Inputs: tensor([[26.0000,  1.0000, 30.0000,  1.0000,  0.0000,  3.0000],
        [30.0000,  1.0000, 35.5300,  0.0000,  1.0000,  2.0000],
        [46.0000,  0.0000, 30.8000,  3.0000,  0.0000,  3.0000],
        [21.0000,  1.0000, 23.7500,  2.0000,  0.0000,  1.0000],
        [40.0000,  0.0000, 29.6000,  0.0000,  0.0000,  3.0000],
        [26.0000,  0.0000, 28.7850,  0.0000,  0.0000,  0.0000],
        [19.0000,  0.0000, 36.5750,  0.0000,  0.0000,  1.0000],
        [60.0000,  1.0000, 28.9000,  0.0000,  0.0000,  3.0000],
        [19.0000,  1.0000, 30.2500,  0.0000,  1.0000,  2.0000],
        [53.0000,  1.0000, 28.8800,  0.0000,  0.0000,  1.0000],
        [34.0000,  1.0000, 25.2700,  1.0000,  0.0000,  1.0000],
        [50.0000,  1.0000, 25.3000,  0.0000,  0.0000,  2.0000],
        [20.0000,  0.0000, 28.7850,  0.0000,  0.0000,  0.0000],
        [43.0000,  0.0000, 35.6400,  1.0000,  0.0000,  2.0000],
        [37.0000,  0.0000, 30.8000,  2.0000,  0.0000,  2.0000],
        [57.0000,  0.0000, 28.78

# Create Linear Regression Model

In [121]:
input_size = len(input_cols)
output_size = len(output_cols)
print(input_size, output_size)

6 1


In [164]:
# Create a model
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, xb):
        out = self.linear(xb)
        return out

    def training_step(self, batch):
        inputs, targets = batch
        # Generate prediction
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        # Generate prediction
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)
        return {'val_loss': loss.detach()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean() # Combine losses
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result, num_epochs):
        # Print result
        print(f"Epoch : [{epoch+1}/{num_epochs}], val_loss: {result['val_loss']:.4f}")

        

In [165]:
model = InsuranceModel()

In [166]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0612, -0.2110, -0.2813, -0.2310,  0.0479, -0.1714]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.3317], requires_grad=True)]

# Train the model and fit the data

In [167]:
def evaluate(model, val_dl):
    outputs = [model.validation_step(batch) for batch in val_dl]
    return model.validation_epoch_end(outputs)

In [168]:
def fit(epochs, lr, model, train_dl, val_dl, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)

    for epoch in range(epochs):
        # Training Phase
        for batch in train_dl:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation Phase
        result = evaluate(model, val_dl) # Get mean loss of each batch
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [169]:
result0 = evaluate(model, val_dl)
print(result0)


{'val_loss': 12860.5146484375}


In [171]:
epochs = 100
lr = 1e-2
h1 = fit(epochs, lr, model, train_dl, val_dl)

Epoch : [1/100], val_loss: 11989.5986
Epoch : [2/100], val_loss: 11618.0283
Epoch : [3/100], val_loss: 11254.8115
Epoch : [4/100], val_loss: 10896.3867
Epoch : [5/100], val_loss: 10542.2510
Epoch : [6/100], val_loss: 10197.8799
Epoch : [7/100], val_loss: 9864.8291
Epoch : [8/100], val_loss: 9549.3779
Epoch : [9/100], val_loss: 9255.3506
Epoch : [10/100], val_loss: 8981.0830
Epoch : [11/100], val_loss: 8727.1143
Epoch : [12/100], val_loss: 8491.6660
Epoch : [13/100], val_loss: 8276.1211
Epoch : [14/100], val_loss: 8074.0469
Epoch : [15/100], val_loss: 7875.7290
Epoch : [16/100], val_loss: 7679.1587
Epoch : [17/100], val_loss: 7484.6841
Epoch : [18/100], val_loss: 7294.1270
Epoch : [19/100], val_loss: 7108.5190
Epoch : [20/100], val_loss: 6920.7231
Epoch : [21/100], val_loss: 6735.2490
Epoch : [22/100], val_loss: 6549.3457
Epoch : [23/100], val_loss: 6363.7944
Epoch : [24/100], val_loss: 6179.8306
Epoch : [25/100], val_loss: 5994.1646
Epoch : [26/100], val_loss: 5806.5640
Epoch : [27/100

# Make a prediction

In [174]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0) # turn to n+1 tensor with parameter 1
    prediction = model(inputs)
    prediction = prediction[0].detach()
    print("Input: ", input)
    print("Target: ", target)
    print("Prediction: ", prediction)

In [175]:
input, target = val_ds[0]
predict_single(input, target, model)

Input:  tensor([22.0000,  1.0000, 26.8400,  0.0000,  0.0000,  2.0000])
Target:  tensor([1664.9996])
Prediction:  tensor([2718.7283])


In [176]:
input, target = val_ds[10]
predict_single(input, target, model)

Input:  tensor([40.0000,  1.0000, 41.2300,  1.0000,  0.0000,  0.0000])
Target:  tensor([6610.1099])
Prediction:  tensor([7681.0352])
